Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu

Boise State University, Department of Chemistry and Biochemistry

# Fitting and Data Analysis for the UV-Vis Particle in a Box Experiment {-}

Before we start, let us import the main modules that we will need for this lecture. You may see some new modules in the list below, we will add more details in the right sections.

In [7]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

We need to load the Google Drive and access an example of a dataset from a calorimetry experiment. You can use the same set that I am using by downloading it from Canvas, [here](). Or you can use your own files. I am assuming the file in question will be located in a `ParticleBox_Data/` subfolder in your `Colab Notebook/` folder. I will be using a file named `particleinabox.csv`, make sure to change this name if you use a different file.

In [4]:
# Load the google drive with your files 
#from google.colab import drive
#drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
#path='/content/drive/MyDrive/Colab Notebooks/Bomb_Data/'
path='ParticleBox_Data/'
file='particleinabox.csv'

The following module needs to be installed on Colab. We won't need it too much for this analysis, but they offer a lot of nice features for chemistry programming. 

In [5]:
#!pip install rdkit
#!pip install cirpy
from rdkit import Chem
from rdkit.Chem import Draw
import cirpy

In particular we can use them to draw the molecules in our experiments

In [8]:
data=pd.read_csv(path+file)
data

,code,p,o,s,lambda
0,977-96-8,3,0,0,523
1,605-91-4,5,0,0,604
2,4727-49-5,7,0,0,590
3,14187-31-6,7,0,0,707
4,4727-50-8,9,0,0,706
5,18300-31-7,11,0,0,814
6,2197-01-5,3,0,1,424
7,905-97-5,5,0,1,557
8,514-73-8,7,0,1,651
9,3071-70-3,9,0,1,681


In [ ]:
BA_smile='O=C(O)c1ccccc1'
BA_img = Draw.MolToImage( Chem.MolFromSmiles(BA_smile), size=(150, 150) )
display(BA_img)
CHex_smile='C1CC(CCC1C(=O)O)C(=O)O'
CHex_img = Draw.MolToImage( Chem.MolFromSmiles(CHex_smile), size=(150, 150) )
display(CHex_img)
CPro_smile='C1CC1C(=O)O'
CPro_img = Draw.MolToImage( Chem.MolFromSmiles(CPro_smile), size=(150, 150) )
display(CPro_img)

We can read the file into a `Pandas.DataFrame` object using the `pd.read_csv()` function. Note that the file in question has three columns and no labels for the columns. The first column contains the time of the experiment in a HH:MM:SS format, the second column contains the temperature recorded by the calorimeter, while the last column contains the time of the experiment in seconds. For what we need we can skip the first column and only read the second and third, but we need to make sure to tell `Pandas` the names of the columns we are reading.

In [ ]:
data=pd.read_csv(path+file,usecols=(1,2),names=['temperature','time'])
data.head()

For later analysis we will need to know the end time of the experiment. There are two alternative ways to access a single cell in a `DataFrame`. The `DataFrame.iloc[i,j]` method allows to access the cells using the same indexing as `Numpy.array`. The `DataFrame.loc['index','column']` approach allows to instead use the row's index and column name.  

In [ ]:
time_end=data.iloc[-1,1]
print(data.iloc[-1,1])
print(data.loc[len(data)-1,'time'])

As usual we can plot the two columns using `Matplotlib.Pyplot` functions.

In [ ]:
plt.plot(data['time'],data['temperature'])
plt.xlabel('Time (s)')
plt.ylabel('Temperature ($^{\circ}$C)')
plt.show()

How can we fit this curve to extract the important information, i.e. the jump in temperature during combustion?

There are three regimes in this curve, a linear drift in temperature in the pre-ignition part, an exponential growth during combustion, and a linear drift (with negative slope) in the post-ignition part. We need to find ways to identify the boundary of the three regions and to fit the individual regimes.

## Finding the Pre-Ignition Temperature {-}

The very first part of the experiment is clearly not as clean and stable as we would need. We can assume there is an initial transient time that we need to wait before being able to use our data. As this initial time is not so critical in the following fitting steps, we can just identify a reasonable starting time from the plot. 

In [ ]:
time_start = 150

However, we need to be very accurate in identifying the point in time at which the combustion starts. At that moment the datapoints start to move away from the linear (almost flat) drifting regime. We could visually inspect our data to identify the point at which the curve starts to grow in a visible fashion. Or we could try to use Python to automate this task. If we perform a linear regression fit of the drift regime the quality of the fit will decrease significantly if we include points after the pre-ignition time.

In [ ]:
time_preignition_guess = 340
preignition_lr=LinearRegression()
preignition_data=data.query('time > {} and time < {}'.format(time_start,time_preignition_guess))
x=preignition_data['time'].values.reshape(-1,1)
y=preignition_data['temperature'].values
preignition_lr.fit(x,y)
plt.plot(x,y)
plt.plot(x,preignition_lr.predict(x))
plt.show()

Using this strategy we can systematically repeat the fit as a function of time and stop when the quality of the fit drops below a certain threshold. 

In [ ]:
times=np.arange(200,400,10.)
preignition_lr=LinearRegression()
score_threshold = 0.8
for time in times:
    preignition_data=data.query('time > {} and time < {}'.format(time_start,time))
    x=preignition_data['time'].values.reshape(-1,1)
    y=preignition_data['temperature'].values
    preignition_lr.fit(x,y)
    if preignition_lr.score(x,y) < score_threshold : 
        time_preignition = time - 10.
        break
print(time_preignition)

We can convert this block of commands into a function, so that they can be used multiple times without the need to rewrite everything. Python functions start with the `def function_name():` statement. Every command in the function needs to be indented. 

In [ ]:
def find_preignition_time(data,time_start,time_post,timestep,threshold=0.8):
    """
    Function to identify the pre-ignition time from calorimetry curves

    Input variables:
        data = a DataFrame with a 'time' and a 'temperature' columns
        time_start = skip the first part of the curve
        time_post = arbitrary time after the ignition
        timestep = time interval between collected datapoints
        threshold = the score at which we decide the curve deviates from linear (optional)

    Output:
        preignition_time = the time at which the curve deviates from a linear drift
    """
    times=np.arange(time_start+5*timestep,time_post,timestep)
    preignition_lr=LinearRegression()
    for time in times:
        preignition_data=data.query('time > {} and time < {}'.format(time_start,time))
        x=preignition_data['time'].values.reshape(-1,1)
        y=preignition_data['temperature'].values
        preignition_lr.fit(x,y)
        if preignition_lr.score(x,y) < threshold : 
            preignition_time = time - 10.
            break
    return preignition_time # this is the quantity that the function passes in output

We can now use the function as any other Python function.

In [ ]:
print(find_preignition_time(data,150,400,10))

## Fitting the Pre-Ignition Drift {-}

Now that we have determined the pre-ignition time, we can repeat the fit on the left part of the curve to get the temperature drift and temperature right before the ignition. 

In [ ]:
preignition_data=data.query('time > {} and time < {}'.format(time_start,time_preignition))
x=preignition_data['time'].values.reshape(-1,1)
y=preignition_data['temperature'].values
preignition_lr.fit(x,y)
temperature_preignition = preignition_lr.predict(np.array([[time_preignition]]))
print("The pre-ignition time is {} s".format(time_preignition))
print("The pre-ignition temperature is {} C".format(temperature_preignition[0]))
print("The pre-ignition drift in temperature is {} C/s".format(preignition_lr.coef_[0]))

We can generate the datas associated with the first linear fit, for plotting the fits together with the experiment

In [ ]:
times_preignition=np.arange(0,500,10.).reshape(-1,1)
temperatures_preignition=preignition_lr.predict(times_preignition)

## Fitting the Post-Ignition Drift {-}

We can choose the post-ignition time as we like, provided it is in the linear drift region of the data. 

In [ ]:
time_postignition=750

We can perform the fit of the post-ignition drift using again a linear regression model. This step is left as an exercise for the student

In [ ]:
postignition_lr = LinearRegression()
postignition_data = 0.0
temperature_postignition = 0.0
print("The post-ignition time is {} s".format(time_postignition))
print("The post-ignition temperature is {} C".format(temperature_postignition))
print("The post-ignition drift in temperature is {} C/s".format(postignition_lr.coef_[0]))

We can generate the datas associated with the second linear fit, for plotting the fits together with the experiment

In [ ]:
times_postignition=np.arange(400,time_end,10.).reshape(-1,1)
temperatures_postignition=postignition_lr.predict(times_postignition)

## Fitting the Exponential Curve {-}

For the combustion regime we want to fit the curve with the function $T(t)=T_i + (T_f-T_i)(1-e^{-kt})$. We can recast this formula into a linear fit if we look at the data as $\frac{T_f-T(t)}{T_f-T_i}=e^{-kt}$ and we fit the natural logarithm of the data with a linear model. For convenience we will only perform the fit from the pre-ignition time and up to a certain time, shorter than the post-ignition time used above. 

In [ ]:
time_exponential = 650
exponential_data=data.query('time > {} and time < {}'.format(time_preignition,time_exponential))
x=exponential_data['time'].values.reshape(-1,1)
y=np.log(temperature_postignition-exponential_data['temperature'].values)
exponential_lr=LinearRegression()
exponential_lr.fit(x,y)
rate = -exponential_lr.coef_[0]
print("The temperature increase rate is {} 1/s".format(rate))
time_determination = time_preignition+1/rate
print("The time at which to determine the temperature difference is {} s".format(time_determination))

We can generate the datas associated with the exponential increase

In [ ]:
times_exponential=np.arange(time_preignition,time_postignition,0.1).reshape(-1,1)
temperatures_exponential=temperature_preignition + (temperature_postignition-temperature_preignition) * (1 - np.exp(rate*(time_preignition-times_exponential))) 

Finally, having determined the time at the middle of the exponential increase we can compute the corrected temperature jump in the experiment

In [ ]:
temperature_preignition_corrected = preignition_lr.predict(np.array([[time_determination]]))[0]
temperature_postignition_corrected = postignition_lr.predict(np.array([[time_determination]]))[0]
deltaT = temperature_postignition_corrected - temperature_preignition_corrected
print("The corresponding DeltaT is {} s".format(deltaT))

## Plotting the Experiments with the Fitting Curves {-}

In [ ]:
plt.plot(data['time'],data['run1'],'ko',ms=2)
plt.plot(times_preignition,temperatures_preignition)
plt.plot(times_postignition,temperatures_postignition)
plt.plot(times_exponential,temperatures_exponential)
plt.xlabel('Time (s)')
plt.ylabel('Temperature ($^{\circ}$C)')
plt.show()